In [ ]:
import os
import sys
import platform
import datetime

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC, SVR

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LOG_LOWER_LIMIT = -7 # np.exp(-7) = 0.00091188196555451624
SAVE_EVALUATIONS = True
# SAVE_FEATURES = False
# SAVE_OUTPUTS = True

In [ ]:
system_name = platform.system()
if system_name == 'Linux':
    github_dir = '/home/gor/codes/dVLogger-Project'
elif system_name == 'Darwin':
    github_dir = '/Users/gor/codes/dVLogger-Project'
features_npz = os.path.join(github_dir, 'data', 'case_sum_0706_A_1.npz')
clinical_outcomes_npz = os.path.join(github_dir, 'data', 'clinical_outcomes_0707.npz')
folds_clinical_outcomes_npz = os.path.join(github_dir, 'data', 'folds_clinical_outcomes_0707.npz')

d = datetime.date.today()
os.makedirs(os.path.join(github_dir, 'results', '{:02d}{:02d}'.format(d.month, d.day)), exist_ok=True)
clf_eval_csv = os.path.join(github_dir,
                            'results',
                            '{:02d}{:02d}'.format(d.month, d.day),
                            'clinical_outcomes_classification_results.csv')
reg_eval_csv = os.path.join(github_dir,
                            'results',
                            '{:02d}{:02d}'.format(d.month, d.day),
                            'clinical_outcomes_regression_results.csv')

In [ ]:
def normalize(x, mean=None, std=None):
    if mean is None:
        mean = x.mean(axis=0)
        std = x.std(axis=0)
        std[std < 1e-9] = 1e-9
    return (x - mean) / std, mean, std

In [ ]:
def as_one_hot(y):
    assert(y.ndim == 1 or y.shape[1] == 1)
    assert(np.isclose(np.min(y), 0))
    n_samples = y.shape[0]
    class_count = np.int32(np.max(y) + 1)
    r = np.zeros((n_samples, class_count), dtype=np.int32)
    r[(np.arange(n_samples), y.astype(np.int32))] = 1
    return r

In [ ]:
clf_list = [
    LogisticRegression('l2'),
    SVC(kernel='rbf', decision_function_shape='ovr'),
    RandomForestClassifier(n_estimators=20),
    RandomForestClassifier(n_estimators=50),
]
clf_name_list = ['LR-L2', 'SVM-RBF', 'RF-20', 'RF-50']
clf_need_norm_list = [False, True, False, False]
clf_eval_list = {
    'Acc': metrics.accuracy_score,
    'Prec': lambda a, b: metrics.precision_score(a, b, average='micro'),
    'Rec': lambda a, b: metrics.recall_score(a, b, average='micro'),
    'AUROC': lambda a, b: metrics.roc_auc_score(a, b, average='micro'),
}
reg_list = [
    LinearRegression('l1'),
    LinearRegression('l2'),
    RandomForestRegressor(n_estimators=20),
    RandomForestRegressor(n_estimators=50),
    # SVR: The free parameters in the model are C and epsilon
#     SVR(kernel='linear'),
#     SVR(kernel='poly'),
#     SVR(kernel='rbf'),
]
reg_name_list = [
    'LiR-L1',
    'LiR-L2',
    'RF-Reg-20',
    'RF-Reg-50',
#     'SVR-Li',
#     'SVR-Poly',
#     'SVR-RBF',
]
reg_need_norm_list = [False] * len(reg_list)
reg_eval_list = {
    'MAE': metrics.mean_absolute_error,
#     'Explained-Variance-Sc': metrics.explained_variance_score,
    'R2-Sc': metrics.r2_score,
}

In [ ]:
locals().update(np.load(features_npz))
locals().update(np.load(clinical_outcomes_npz))
locals().update(np.load(folds_clinical_outcomes_npz))

In [ ]:
assert(np.all(X_case == Y_case))
case = X_case
n_splits = folds.shape[1]

In [ ]:
if has_dev_set:
    data_set_list = ['train', 'dev', 'test']
else:
    data_set_list = ['train', 'test']

In [ ]:
if SAVE_EVALUATIONS:
    header = ['y_name', 'model_name']
    for p1 in data_set_list:
        for p3 in clf_eval_list.keys():
            for p2 in ['mean', 'std']:
                header.append('_'.join([p1, p2, p3]))
    with open(clf_eval_csv, 'w') as f:
        f.write(','.join(header) + '\n')
    
    header = ['y_name', 'model_name']
    for p1 in data_set_list:
        for p3 in reg_eval_list.keys():
            for p2 in ['mean', 'std']:
                header.append('_'.join([p1, p2, p3]))
    with open(reg_eval_csv, 'w') as f:
        f.write(','.join(header) + '\n')

In [ ]:
Y_col

In [ ]:
for y_idx in range(len(Y_col)):
    y = Y[:, y_idx]
    y_name = Y_col[y_idx]
    y_train_task_type = Y_train_task_type[y_idx]
    print(y_name)

    if y_train_task_type == 'classification':
        model_list = clf_list
        model_name_list = clf_name_list
        model_need_norm_list = clf_need_norm_list
        model_eval_list = clf_eval_list
    elif y_train_task_type == 'regression':
        model_list = reg_list
        model_name_list = reg_name_list
        model_need_norm_list = reg_need_norm_list
        model_eval_list = reg_eval_list
    else:
        raise ValueError

    for model_idx in range(len(model_list)):
        model = model_list[model_idx]
        model_name = model_name_list[model_idx]
        model_need_norm = model_need_norm_list[model_idx]
        print('{} : '.format(model_name), end='')

        eval_train, eval_test = {}, {}
        for eval_name in model_eval_list:
            eval_train[eval_name] = []
            eval_test[eval_name] = []
        if has_dev_set:
            eval_dev = {}
            for eval_name in model_eval_list:
                eval_dev[eval_name] = []

        for split_idx in range(n_splits):
            train_idx = folds[y_idx, split_idx, 0]
            if has_dev_set:
                dev_idx = folds[y_idx, split_idx, 1]
                test_idx = folds[y_idx, split_idx, 2]
            else:
                test_idx = folds[y_idx, split_idx, 1]
            if model_need_norm:
                x_train, x_mean, x_std = normalize(X[train_idx, :])
                if has_dev_set:
                    x_dev, _, _ = normalize(X[dev_idx, :], x_mean, x_std)
                x_test, _, _ = normalize(X[test_idx, :], x_mean, x_std)
            else:
                x_train = X[train_idx, :]
                if has_dev_set:
                    x_dev = X[dev_idx, :]
                x_test = X[test_idx, :]
            y_train = Y[train_idx, y_idx]
            if has_dev_set:
                y_dev = Y[dev_idx, y_idx]
            y_test = Y[test_idx, y_idx]

            print('{}'.format(split_idx + 1), end='')
            model.fit(x_train, y_train)
            y_pred_train = model.predict(x_train)
            if has_dev_set:
                y_pred_dev = model.predict(x_dev)
            y_pred_test = model.predict(x_test)
            if y_train_task_type == 'classification':
                if hasattr(model, 'predict_proba'):
                    y_score_train = model.predict_proba(x_train)
                    if has_dev_set:
                        y_score_dev = model.predict_proba(x_dev)
                    y_score_test = model.predict_proba(x_test)
                else:
                    y_score_train = model.decision_function(x_train)
                    if has_dev_set:
                        y_score_dev = model.decision_function(x_dev)
                    y_score_test = model.decision_function(x_test)

            if '_log_' in y_name:
                y_train = np.exp(y_train)
                y_pred_train[y_pred_train < LOG_LOWER_LIMIT] = LOG_LOWER_LIMIT
                y_pred_train = np.exp(y_pred_train)
                if has_dev_set:
                    y_dev = np.exp(y_dev)
                    y_pred_dev[y_pred_dev < LOG_LOWER_LIMIT] = LOG_LOWER_LIMIT
                    y_pred_dev = np.exp(y_pred_dev)
                y_test = np.exp(y_test)
                y_pred_test[y_pred_test < LOG_LOWER_LIMIT] = LOG_LOWER_LIMIT
                y_pred_test = np.exp(y_pred_test)
            
            print(', ', end='')
            for eval_name, eval_fn in model_eval_list.items():
                if eval_name == 'AUROC':
                    if y_score_train.ndim == 1:
                        eval_train[eval_name].append(eval_fn(y_train, y_score_train))
                        if has_dev_set:
                            eval_dev[eval_name].append(eval_fn(y_dev, y_score_dev))
                        eval_test[eval_name].append(eval_fn(y_test, y_score_test))
                    else:
                        eval_train[eval_name].append(eval_fn(as_one_hot(y_train), y_score_train))
                        if has_dev_set:
                            eval_dev[eval_name].append(eval_fn(as_one_hot(y_dev), y_score_dev))
                        eval_test[eval_name].append(eval_fn(as_one_hot(y_test), y_score_test))
                else:
                    eval_train[eval_name].append(eval_fn(y_train, y_pred_train))
                    if has_dev_set:
                        eval_dev[eval_name].append(eval_fn(y_dev, y_pred_dev))
                    eval_test[eval_name].append(eval_fn(y_test, y_pred_test))
        print()

        for eval_name in model_eval_list:
            if has_dev_set:
                print('{:21s} :: Trn: {:.3f} ({:.3f}), Dev: {:.3f} ({:.3f}), Tst: {:.3f} ({:.3f})'.format(
                    eval_name,
                    np.mean(eval_train[eval_name]), np.std(eval_train[eval_name]),
                    np.mean(eval_dev[eval_name]), np.std(eval_dev[eval_name]),
                    np.mean(eval_test[eval_name]), np.std(eval_test[eval_name])))
            else:
                print('{:21s} :: Trn: {:.3f} ({:.3f}), Tst: {:.3f} ({:.3f})'.format(
                    eval_name,
                    np.mean(eval_train[eval_name]), np.std(eval_train[eval_name]),
                    np.mean(eval_test[eval_name]), np.std(eval_test[eval_name])))
            
        if SAVE_EVALUATIONS:
#             header = ['y_name', 'model_name']
#             for p1 in ['train', 'dev', 'test']:
#                 for p3 in reg_eval_list.keys():
#                     for p2 in ['mean', 'std']:
#                         header.append('_'.join([p1, p2, p3]))
            row = [y_name, model_name]
            for p1 in data_set_list:
                for p3 in model_eval_list.keys():
                    for p2 in ['mean', 'std']:
                        row.append(eval('"{:.3f}".format(np.' + p2 + '(eval_' + p1 + '["' + p3 + '"]))'))
            if y_train_task_type == 'classification':
                with open(clf_eval_csv, 'a') as f:
                    f.write(','.join(row) + '\n')
            elif y_train_task_type == 'regression':
                with open(reg_eval_csv, 'a') as f:
                    f.write(','.join(row) + '\n')
            else:
                raise ValueError
    
    print()